## Questions
<ul>
<li>What range of years does the provided database cover?</li>
<br> 
<li>Find the name and height of the shortest player in the database. How many games did he play in? What is the name of the team for which he played?</li>
<br> 
<li>Find all players in the database who played at Vanderbilt University. Create a list showing each player’s first and last names as well as the total salary they earned in the major leagues. Sort this list in descending order by the total salary earned. Which Vanderbilt player earned the most money in the majors?</li>
<br> 
<li>Using the fielding table, group players into three groups based on their position: label players with position OF as "Outfield", those with position "SS", "1B", "2B", and "3B" as "Infield", and those with position "P" or "C" as "Battery". Determine the number of putouts made by each of these three groups in 2016.</li>
<br> 
<li>Find the average number of strikeouts per game by decade since 1920. Round the numbers you report to 2 decimal places. Do the same for home runs per game. Do you see any trends?</li>
<br> 
<li>Find the player who had the most success stealing bases in 2016, where success is measured as the percentage of stolen base attempts which are successful. (A stolen base attempt results either in a stolen base or being caught stealing.) Consider only players who attempted at least 20 stolen bases.</li>
<br> 
<li>From 1970 – 2016, what is the largest number of wins for a team that did not win the world series? What is the smallest number of wins for a team that did win the world series? Doing this will probably result in an unusually small number of wins for a world series champion – determine why this is the case. Then redo your query, excluding the problem year. How often from 1970 – 2016 was it the case that a team with the most wins also won the world series? What percentage of the time?</li>
<br> 
<li>Using the attendance figures from the homegames table, find the teams and parks which had the top 5 average attendance per game in 2016 (where average attendance is defined as total attendance divided by number of games). Only consider parks where there were at least 10 games played. Report the park name, team name, and average attendance. Repeat for the lowest 5 average attendance.</li>
<br> 
<li>Which managers have won the TSN Manager of the Year award in both the National League (NL) and the American League (AL)? Give their full name and the teams that they were managing when they won the award.</li> 
</ul>

## Bonus Questions
<ul>
<li>Analyze all the colleges in the state of Tennessee. Which college has had the most success in the major leagues. Use whatever metric for success you like - number of players, number of games, salaries, world series wins, etc.</li>
<br>
<li>Is there any correlation between number of wins and team salary? Use data from 2000 and later to answer this question. As you do this analysis, keep in mind that salaries across the whole league tend to increase together, so you may want to look on a year-by-year basis.</li>
<br>
<li>It is thought that since left-handed pitchers are more rare, causing batters to face them less often, that they are more effective. Investigate this claim and present evidence to either support or dispute this claim. First, determine just how rare left-handed pitchers are compared with right-handed pitchers. Are left-handed pitchers more likely to win the Cy Young Award? Are they more likely to make it into the hall of fame?</li>

In [58]:
import os

import pandas as pd
from sqlalchemy import create_engine
import numpy as np

In [59]:
from dotenv import load_dotenv
load_dotenv()

True

In [60]:
username = os.getenv('POSTGRES_USERNAME')
password = os.getenv('POSTGRES_PASSWORD')


postgres_connection_string = "postgres://{username}:{password}@{host}:{port}/{database}?gssencmode=disable".format(
    username=username,
    password=password,
    host="localhost",
    port="5432",
    database="lahman_baseball"
)

In [61]:
engine = create_engine(postgres_connection_string)

In [89]:
batting_sql = "SELECT * FROM batting;"
teams_sql = "SELECT * FROM teams;"
people_sql = "SELECT * FROM people;"
appearances_sql = "SELECT * FROM appearances;"
collegeplaying_sql = "SELECT * FROM collegeplaying;"
schools_sql = "SELECT * FROM schools;"
salaries_sql = "SELECT * FROM salaries;"

In [90]:
batting_df = pd.read_sql(batting_sql, con=engine)
teams_df = pd.read_sql(teams_sql, con=engine)
people_df = pd.read_sql(people_sql, con=engine)
appearances_df = pd.read_sql(appearances_sql, con=engine)
collegeplaying_df = pd.read_sql(collegeplaying_sql, con=engine)
schools_df = pd.read_sql(schools_sql, con=engine)
salaries_df = pd.read_sql(salaries_sql, con=engine)

#### What range of years does the provided database cover?

In [64]:
teams_df['yearid'].min()

1871

In [65]:
teams_df['yearid'].max()

2016

#### Find the name and height of the shortest player in the database. How many games did he play in? What is the name of the team for which he played?

In [66]:
shortest_player = people_df[people_df.height == people_df.height.min()] 

In [67]:
shortest_player['namegiven']

5843    Edward Carl
Name: namegiven, dtype: object

In [68]:
edward_carl_id = list(shortest_player['playerid'])

In [69]:
appearances_df[appearances_df['playerid'].isin(['gaedeed01'])]

,yearid,teamid,lgid,playerid,g_all,gs,g_batting,g_defense,g_p,g_c,...,g_2b,g_3b,g_ss,g_lf,g_cf,g_rf,g_of,g_dh,g_ph,g_pr
35173,1951,SLA,AL,gaedeed01,1,0.0,1,0,0,0,...,0,0,0,0,0,0,0,0.0,1.0,0.0


In [70]:
teams_df['name'][teams_df['teamid'].isin(['SLA'])].head(1)

412    St. Louis Browns
Name: name, dtype: object

#### Find all players in the database who played at Vanderbilt University. Create a list showing each player’s first and last names as well as the total salary they earned in the major leagues. Sort this list in descending order by the total salary earned. Which Vanderbilt player earned the most money in the majors?

In [81]:
vandy_id = list(schools_df.loc[schools_df['schoolname'] == 'Vanderbilt University', 'schoolid'])

In [82]:
vandy_players = list(collegeplaying_df.loc[collegeplaying_df['schoolid'] == 'vandy', 'playerid'])

In [83]:
# function to get unique values 
def unique(list1): 
    x = np.array(list1) 
    print(np.unique(x)) 

In [84]:
unique(vandy_players)

['alvarpe01' 'baxtemi01' 'chrisni01' 'colliwi01' 'corajo01' 'embresl01'
 'flahery01' 'grayso01' 'hendrha01' 'katama01' 'lewisje01' 'madissc01'
 'minormi01' 'mooresc01' 'mossma01' 'pauljo01' 'priceda01' 'priorma01'
 'richaan01' 'sandesc01' 'sewelri01' 'sowerje01' 'willimi01' 'zeidjo01']


In [87]:
#Placed a comma between all items returned above.
vandy_players_unique = ['alvarpe01', 'baxtemi01', 'chrisni01', 'colliwi01', 'corajo01', 'embresl01',
 'flahery01', 'grayso01', 'hendrha01', 'katama01', 'lewisje01', 'madissc01',
 'minormi01', 'mooresc01', 'mossma01', 'pauljo01', 'priceda01', 'priorma01',
 'richaan01', 'sandesc01', 'sewelri01', 'sowerje01', 'willimi01', 'zeidjo01']

In [96]:
vandy_names = people_df.query('playerid in @vandy_players_unique')

In [126]:
vandy_names_fl = vandy_names[['playerid', 'namefirst', 'namelast']]

In [127]:
vandy_names_fl

,playerid,namefirst,namelast
282,alvarpe01,Pedro,Alvarez
958,baxtemi01,Mike,Baxter
2989,chrisni01,Nick,Christiani
3292,colliwi01,Wilson,Collins
3466,corajo01,Joey,Cora
4990,embresl01,Slim,Embry
5457,flahery01,Ryan,Flaherty
6605,grayso01,Sonny,Gray
7467,hendrha01,Harvey,Hendrick
8861,katama01,Matt,Kata


In [121]:
vandy_salaries = salaries_df.query('playerid in @vandy_players_unique')

In [128]:
vandy_total_salaries = vandy_salaries.groupby('playerid')['salary'].sum().to_frame()

In [133]:
pd.merge(vandy_total_salaries, vandy_names_fl, on = 'playerid').sort_values(by = 'salary', ascending = False)

,playerid,salary,namefirst,namelast
11,priceda01,81851296.0,David,Price
0,alvarpe01,20681704.0,Pedro,Alvarez
12,priorma01,12800000.0,Mark,Prior
13,sandesc01,10750000.0,Scott,Sanderson
9,minormi01,6837500.0,Mike,Minor
3,corajo01,5622500.0,Joey,Cora
4,flahery01,4061000.0,Ryan,Flaherty
10,pauljo01,2640000.0,Josh,Paul
1,baxtemi01,2094418.0,Mike,Baxter
5,grayso01,1542500.0,Sonny,Gray
